# Inference and Evaluation with the Finetuned Models

In this notebook, we demonstrate how one can evaluate various finetuned models.

The main steps are:
  * Preparing the models and datasets
  * Running inference and collecting the results for each dataset


## Setting Up the Environment

While ProkBERT can operate on CPUs, leveraging GPUs significantly accelerates the process. Google Colab offers free GPU usage (subject to time and memory limits), making it an ideal platform for trying and experimenting with ProkBERT models.


### Enabling and testing the GPU (if you are using google colab)

First, you'll need to enable GPUs for the notebook:

- Navigate to Edit→Notebook Settings
- select GPU from the Hardware Accelerator drop-down



### Setting up the packages and the installs
First, we'll install the ProkBERT package directly from its GitHub repository:


In [16]:
# ProkBERT
#!pip install datasets

from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import matthews_corrcoef, accuracy_score, roc_auc_score, recall_score
from datasets import load_dataset
import torch
import os


Next, we'll confirm that we can connect to the GPU with pytorch:

In [17]:
# Check if CUDA (GPU support) is available
if not torch.cuda.is_available():
    raise SystemError('GPU device not found')
else:
    device_name = torch.cuda.get_device_name(0)
    print(f'Found GPU at: {device_name}')
num_cores = os.cpu_count() 
print(f'Number of available CPU cores: {num_cores}')    

Found GPU at: Quadro RTX 6000
Number of available CPU cores: 96


## Loading the Finetuned Model and the tokenizer for Promoter Identification

Next, we will download the finetuned model for promoter identification. For more details about the model, please see the Finetuning notebook.



In [18]:

finetuned_model = "neuralbioinfo/prokbert-mini-promoter"
tokenizer = AutoTokenizer.from_pretrained(finetuned_model, trust_remote_code=True)
model = AutoModelForSequenceClassification.from_pretrained(finetuned_model, trust_remote_code=True)



A new version of the following files was downloaded from https://huggingface.co/neuralbioinfo/prokbert-mini-promoter:
- tokenizer.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


models.py:   0%|          | 0.00/11.6k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/neuralbioinfo/prokbert-mini:
- models.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


model.safetensors:   0%|          | 0.00/82.6M [00:00<?, ?B/s]

## Loading the Dataset

In this section, we will evaluate the test sets of the promoter dataset. We have two different types of tests: a sigma70 test (known E. coli promoters) referred to as the 'test_sigma70' set, and a multispecies dataset, which consists of promoters from various species as well as CDS sequences (non-promoters) and randomly generated sequences. For a more detailed description, see: [Bacterial Promoters Dataset](https://huggingface.co/datasets/neuralbioinfo/bacterial_promoters)



In [ ]:
def tokenize_function(examples):
    # Tokenize the input sequences
    encoded = tokenizer.batch_encode_plus(
        examples["segment"],
        padding=True,
        add_special_tokens=True,
        return_tensors="pt",
    )
    # Return the updated dictionary
    return encoded
    
dataset = load_dataset("neuralbioinfo/bacterial_promoters", split='test_sigma70')
tokenized_dataset = dataset.map(tokenize_function, batched=True, num_proc=num_cores)





In [23]:
from transformers import Trainer, TrainingArguments

# Define TrainingArguments for inference
training_args = TrainingArguments(
    output_dir="./results",              # Directory for storing logs/results
    per_device_eval_batch_size=128,     # Batch size for inference
    logging_dir="./logs",               # Directory for logs
    report_to="none",                   # Disable reporting to W&B or other loggers
)

# Initialize Trainer for inference
trainer = Trainer(
    model=model,                        # Pretrained ProkBERT model
    args=training_args,                 # Inference arguments
)

# Perform inference
predictions = trainer.predict(tokenized_dataset)


2025-01-09 16:11:59,469 - WARNING - Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [29]:
import pandas as pd
import numpy as np
from scipy.special import softmax

probabilities = softmax(predictions.predictions, axis=1)
predicted_labels = np.argmax(probabilities, axis=1)
dataset_df = dataset.to_pandas()
dataset_df["prob_class_0"] = probabilities[:, 0]
dataset_df["prob_class_1"] = probabilities[:, 1]
dataset_df["predicted_label"] = predicted_labels

import ace_tools as tools; tools.display_dataframe_to_user(name="ProkBERT Inference Results", dataframe=dataset_df)


ModuleNotFoundError: No module named 'ace_tools'

In [30]:
dataset_df

,segment_id,ppd_original_SpeciesName,Strand,segment,class_label,L,prom_class,y,prob_class_0,prob_class_1,predicted_label
0,sigma70s_neg00000000,nan,nan,CAAGTCTTCATGTGAAGCAGAGAGCTACTATTAATATCGACGGTGT...,non_promoter,81,sigma70s_neg,0,0.344011,0.655989,1
1,sigma70s_neg00000001,nan,nan,GTACTTATGAGTAATTTACATAAACAATCTGACCGGACTGAATGTC...,non_promoter,81,sigma70s_neg,0,0.770286,0.229714,0
2,sigma70s_neg00000002,nan,nan,GACGGACCCTTATTCACTTACGAACGCATTTCTTCTCTTGTATTGG...,non_promoter,81,sigma70s_neg,0,0.831560,0.168440,0
3,sigma70s_neg00000003,nan,nan,ATACAATTTGTACACTGGAAAATCACAAGTAAAGGAAGGTCGCAAC...,non_promoter,81,sigma70s_neg,0,0.859872,0.140128,0
4,sigma70s_neg00000004,nan,nan,TGTGCGTTGATGCCTCTATTTGTATGGAACAATGTTCCTGAATGAT...,non_promoter,81,sigma70s_neg,0,0.302382,0.697618,1
...,...,...,...,...,...,...,...,...,...,...,...
1859,sigma70s_pos00000860,nan,nan,CGGTTGCCAACAACGTTCATAACTTTGTTGAGCACCGATACGCATT...,promoter,81,sigma70s_pos,1,0.530628,0.469372,0
1860,sigma70s_pos00000861,nan,nan,TGCTCTCGTTTCCTAAGAGTTGTTGCATTTTGCTATATGTTACAAT...,promoter,81,sigma70s_pos,1,0.095307,0.904693,1
1861,sigma70s_pos00000862,nan,nan,CTCGCTGTATCTCTGATAAAACTTGACTCTGGAGTCGACTCCAGAG...,promoter,81,sigma70s_pos,1,0.079808,0.920192,1
1862,sigma70s_pos00000863,nan,nan,TATGTAACATAATGCGACCAATAATCGTAATGAATATGAGAAGTGT...,promoter,81,sigma70s_pos,1,0.095940,0.904060,1


2025-01-09 16:10:00,770 - WARNING - Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


PredictionOutput(predictions=array([[-0.36701283,  0.27845627],
       [ 0.632904  , -0.5770252 ],
       [ 0.81924   , -0.77748686],
       ...,
       [-1.2759869 ,  1.168967  ],
       [-1.1626655 ,  1.0805084 ],
       [-0.9650734 ,  0.84178895]], dtype=float32), label_ids=None, metrics={'test_model_preparation_time': 0.0028, 'test_runtime': 1.0827, 'test_samples_per_second': 1721.666, 'test_steps_per_second': 13.855})

Map (num_proc=96):   0%|          | 0/1864 [00:00<?, ? examples/s]

In [22]:

# Set the evaulation parameters, adjust the batch size if needed?

training_args = TrainingArguments(
    output_dir="./results",
    per_device_eval_batch_size=128,
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args
)
predictions = trainer.predict(tokenized_dataset)





2025-01-09 16:10:57,844 - WARNING - Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


### Tokenization and PyTorch Dataset Creation

Now that we have the tokenizer, the dataset can be prepared for evaluation. Here, we prepare the data for both sigma70 and multispecies datasets.

#### Creating Datasets

We will process the data into a format suitable for our PyTorch model. This involves tokenizing the sequences and converting them into a format that our model can understand. The following code will convert our `test_ms_db` (multispecies dataset) and `test_sigma70_db` (sigma70 dataset) into PyTorch datasets.


In [ ]:
## Creating datasets!
[X_test, y_test, torchdb_test] = get_torch_data_from_segmentdb_classification(tokenizer, test_ms_db)
print(f'Processing validation database!')
[X_val, y_val, torchdb_val] = get_torch_data_from_segmentdb_classification(tokenizer, test_sigma70_db)

test_ds = ProkBERTTrainingDatasetPT(X_test, y_test, AddAttentionMask=True)
val_ds = ProkBERTTrainingDatasetPT(X_val, y_val, AddAttentionMask=True)


## Performing Inference and Evaluation

Now that both the model and the data are prepared, we will perform inference and process the dataset.

For simplicity, this demonstration will be conducted on a single GPU. An important parameter to consider is the batch size, which is set to 1024 in our example. Adjust this value according to the capabilities of your GPU to ensure efficient processing.

In real-world scenarios, especially when evaluating large datasets (exceeding 1,000,000 samples), we recommend utilizing Torch Distributed Data Parallel (DDP) and compiled models for optimized performance. 

The prediction results will be aggregated into a list of numpy arrays, each containing the predicted label, ground truth label, and logits for each class (promoter vs. non-promoter). The evaluation metrics, including AUC, Matthews Correlation Coefficient (MCC), accuracy, and others, will be summarized and returned in a dictionary format.

Below is the code to perform batch-wise inference and accumulate the prediction results:


In [ ]:
batch_size = 4096
dataloader = DataLoader(test_ds, batch_size=batch_size)

# Calculate the total number of batches
total_batches = len(dataloader)

pred_results_ls = []
processed_batches = 0

for batch in dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}  # Move batch data to the appropriate device
    with torch.no_grad():  # Inference mode: no gradient computation
        outputs = model(**batch)
    pred_results = evaluate_binary_classification_bert_build_pred_results(outputs['logits'], batch['labels'])
    pred_results_ls.append(pred_results)  # Collecting prediction results
    
    processed_batches += 1  # Increment the count of processed batches
    percent_complete = (processed_batches / total_batches) * 100  # Calculate the percentage of completion
    print(f'Batch {processed_batches}/{total_batches} processed, {percent_complete:.2f}% complete.')  # Print the progress

# Combine all batch results into one array for evaluation
pred_results = np.concatenate(pred_results_ls)
# Calculate and retrieve evaluation metrics
eval_results, eval_results_ls = evaluate_binary_classification_bert(pred_results)

# cleanup
del model
del batch
del dataset
torch.cuda.empty_cache()


# Convert the results dictionary to a DataFrame
results_df = pd.DataFrame([eval_results])
# Set more meaningful index name
results_df.index = ['Metrics']
results_df


# Inference with the Phage Models

In this section, we will walk through the evaluation of the phage test dataset, following a similar procedure to the previous example. The workflow includes:
  * Preparing the model
  * Preparing the dataset
  * Evaluating the test set and measuring performance metrics


# Prearing the model: 
This is a simple model, that is trained using the 'MegatronBertForSequenceClassification' class.


In [ ]:
from transformers import MegatronBertForSequenceClassification

finetuned_model = "neuralbioinfo/prokbert-mini-phage"
kmer = 6
shift= 1

tok_params = {'kmer' : kmer,
             'shift' : shift}
tokenizer = ProkBERTTokenizer(tokenization_params=tok_params)
model = MegatronBertForSequenceClassification.from_pretrained(finetuned_model)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
model = model.to(device)
_=model.eval()


## Preparing the Phage Dataset for Evaluation

In this section, we load and prepare the phage dataset for evaluation. The dataset is a smaller subset suitable for testing, named "phage-test-small". We will demonstrate how to load this dataset, convert it to a pandas DataFrame.

First, we load the dataset and set the batch size for the DataLoader, which controls how many samples will be processed simultaneously. Then, we convert the loaded dataset to a pandas DataFrame for easier manipulation and processing. Finally, we prepare the data for PyTorch by tokenizing and converting it into a format suitable for our model.


In [ ]:
dataset = load_dataset("nerualbioinfo/phage-test-small")
batch_size = 64

# Loading and converting the dataset
test_set = dataset["sample_test_L1024"].to_pandas()
print(f'Processing the database!')
[X, y, torchdb] = get_torch_data_from_segmentdb_classification(tokenizer, test_set)
dataset = ProkBERTTrainingDatasetPT(X, y, AddAttentionMask=True)
dataloader = DataLoader(dataset, batch_size=batch_size)


## Inference and Evaluation of the Phage Dataset

In this part of the process, we will perform inference on the phage dataset using the prepared DataLoader. The objective is to process each batch from the dataset, make predictions using the finetuned model, and then compile these predictions for evaluation.

We initiate an empty list, `pred_results_ls`, to store the prediction results from each batch. We iterate over each batch in the DataLoader, ensuring that the batch data is moved to the same device as the model (GPU or CPU). With gradient computation disabled (to enhance performance and reduce memory usage during inference), we pass the batch through the model to obtain output logits.

For each batch's output, we evaluate the binary classification results using the `evaluate_binary_classification_bert_build_pred_results` function, which processes the model's logits and the actual labels to generate prediction results. These results are then appended to our list.

After processing all batches, we concatenate the list of arrays into a single numpy array. This aggregated result allows us to evaluate the overall performance of the model on the entire test set. We use the `evaluate_binary_classification_bert` function to calculate various evaluation metrics such as accuracy, precision, recall, F1 score, etc., based on the compiled prediction results. The `eval_results` dictionary will store these metrics for review and analysis.


In [ ]:
# Calculate the total number of batches
total_batches = len(dataloader)

pred_results_ls = []
processed_batches = 0

for batch in dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}  # Move batch data to the appropriate device
    with torch.no_grad():  # Inference mode: no gradient computation
        outputs = model(**batch)
    pred_results = evaluate_binary_classification_bert_build_pred_results(outputs['logits'], batch['labels'])
    pred_results_ls.append(pred_results)  # Collecting prediction results
    
    processed_batches += 1  # Increment the count of processed batches
    percent_complete = (processed_batches / total_batches) * 100  # Calculate the percentage of completion
    print(f'Batch {processed_batches}/{total_batches} processed, {percent_complete:.2f}% complete.')  # Print the progress

# Combine all batch results into one array for evaluation
pred_results = np.concatenate(pred_results_ls)
# Calculate and retrieve evaluation metrics
eval_results, eval_results_ls = evaluate_binary_classification_bert(pred_results)

# Convert the results dictionary to a DataFrame
results_df = pd.DataFrame([eval_results])
# Set more meaningful index name
results_df.index = ['Metrics']
results_df


## Final Remarks

In this evaluation, we have successfully executed the inference process using the pre-trained phage model and assessed its performance on the test dataset. Key metrics such as accuracy, F1 score, Matthews Correlation Coefficient (MCC), and Area Under the Curve (AUC) provide a comprehensive view of the model's ability to distinguish between classes.

Additionally, while these results are promising, further validation and testing on independent datasets are recommended to ensure the model's generalizability and robustness. This could involve cross-validation, additional external datasets, or real-world applications.

Finally, this notebook serves as a foundation for further exploration and refinement. Researchers are encouraged to experiment with different models, parameters, and datasets to enhance understanding and performance. The ultimate goal is to leverage these computational tools to advance our knowledge in microbiology and related fields.
